In [1]:
import pandas as pd
import os
import json
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve
from imblearn.over_sampling import SMOTE

# Carregamento dos dados PPG
print(f"\n[Info] Centralizando dados dos participantes...")
base_path = r'C:\Users\ferna\Documents\Unisinos\Cadeiras\TCC 2\Dataset utilizado\GalaxyPPG\Dataset'

# Listar manual com os IDs dos participantes escolhidos
selected_ids = ['P11', 'P12', 'P13', 'P14', 'P15', 'P17', 'P18', 'P19', 'P20']

# Listar para armazenar os DataFrames
dataframes = []

# Iterar sobre os IDs e carrega cada arquivo
for participant_id in selected_ids:
    ppg_path = os.path.join(base_path, participant_id, 'GalaxyWatch', 'PPG.csv')
    
    if os.path.exists(ppg_path):
        df = pd.read_csv(ppg_path)
        df['participant_id'] = participant_id  # Adiciona a origem dos dados
        dataframes.append(df)
    else:
        print(f"[Aviso] Arquivo não encontrado: {ppg_path}")

# Concatenar todos os dados em um único DataFrame
df_centralizado = pd.concat(dataframes, ignore_index=True)

# Exibir resultado
print(f"[Info] Dados centralizados com sucesso. Total de registros: {len(df_centralizado)}!")
print(df_centralizado.head())

ppg_signal = df_centralizado['ppg'].values     # Extrai o sinal PPG (fotopletismografia)
ppg_ts = df_centralizado['timestamp'].values   # Extrai os timestamps (não usado diretamente)
fs = 25  # Frequência de amostragem do sinal (25 Hz)

# Filtragem do sinal PPG
print(f"\n[Info] Aplicando filtro passa-baixa Butterworth para remover ruído de alta frequência dos sinais PPG...")

def butter_lowpass(data, cutoff=5, fs=25, order=3):
    nyq = 0.5 * fs  # Frequência de Nyquist
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    y = signal.filtfilt(b, a, data)  # Filtro passa-baixa aplicado em modo zero-phase (sem atraso)
    return y

ppg_filtered = butter_lowpass(ppg_signal, cutoff=5, fs=fs, order=3)

# Função para extrair features HRV a partir dos picos detectados
def extract_hrv_features_from_peaks(peaks, fs):
    try:
        if len(peaks) < 3:
            raise ValueError("Poucos picos detectados para análise HRV")

        ibis = np.diff(peaks) / fs
        ibis_ms = ibis * 1000

        rmssd = np.sqrt(np.mean(np.square(np.diff(ibis_ms))))
        sdnn = np.std(ibis_ms)
        pnn50 = np.sum(np.abs(np.diff(ibis_ms)) > 50) / len(ibis_ms)
        mean_ibi = np.mean(ibis_ms)
        iqr_ibi = np.percentile(ibis_ms, 75) - np.percentile(ibis_ms, 25)

        return [rmssd, sdnn, pnn50, mean_ibi, iqr_ibi]
    except Exception as e:
        print(f"Erro no cálculo HRV: {e}")
        return [np.nan] * 5

# Extração das features com janela móvel
window_size = 30 * fs
step_size = 10 * fs

features = []
labels = []

print(f"\n[Info] Extraindo features HRV...")

print(f"\n[Info] Definindo rótulo de anormalidade cardíaca...")

for start in range(0, len(ppg_filtered) - window_size, step_size):
    segment = ppg_filtered[start:start + window_size]
    peaks, _ = find_peaks(segment, distance=int(fs * 0.5))
    if len(peaks) < 3:
        continue
    hrv = extract_hrv_features_from_peaks(peaks, fs)
    if np.any(np.isnan(hrv)):
        continue
    rmssd, sdnn = hrv[0], hrv[1]
    label = int((rmssd < 20) or (sdnn < 50))
    features.append(hrv)
    labels.append(label)

X = np.array(features)
y = np.array(labels)

# Normalização e divisão treino-validação-teste
print(f"\n[Info] Normalizando as features...")
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

print(f"\n[Info] Separando dados em treino (60%) e teste (40%)...")
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.4, random_state=42, stratify=y)

print(f"\n[Info] Aplicando SMOTE apenas no conjunto de treino...")
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

print(f"[Info] Após SMOTE - Classe 0: {sum(y_train == 0)}, Classe 1: {sum(y_train == 1)}")

# Treinamento do modelo MLP com partial_fit
CLASSES = np.array([0, 1])

# Criar modelo MLP
model = MLPClassifier(
    hidden_layer_sizes=(200, 100, 50, 25), # Tupla definindo o número de neurônios em cada camada oculta. Aqui, 4 camadas ocultas com tamanhos variados
    activation='tanh',                     # Função de ativação usada nos neurônios 
    solver='adam',                         # Algoritmo de otimização usado para ajustar pesos (adam = Estimativa Adaptativa de Momentos).
    learning_rate_init=0.005,              # Taxa de aprendizado inicial para o otimizador (quanto maior, mais rápido o ajuste, mas pode ser instável).
    alpha=0.001,                           # Parâmetro de regularização L2 para evitar overfitting (penaliza pesos grandes).
    max_iter=1000,                         # Número máximo de iterações (epochs) por chamada de fit.
    warm_start=True,                       # Permite continuar o treinamento a partir do estado anterior (importante para treino incremental).
    n_iter_no_change=10,                   # Número de iterações sem melhora para parar o treino (não muito usado aqui, pois max_iter=1).
    random_state=42
)

# Treinamento
print(f"\n[Info] Treinando modelo com {len(X_train)} exemplos...")
model.fit(X_train, y_train)

# Salvar modelo final em arquivo JSON
MODEL_FILE = 'Modelo_Centralizado_MLP.json'
print(f"\n[Nuvem] Salvando modelo final...")
final_model_data = {
    "class": model.__class__.__name__,
    "params": model.get_params(),
    "coef": [layer.tolist() for layer in model.coefs_],
    "intercept": [b.tolist() for b in model.intercepts_]
}
with open(MODEL_FILE, 'w') as f:
    json.dump(final_model_data, f)
print(f"[Nuvem] Modelo salvo em '{MODEL_FILE}'.")

# Avaliação do modelo
print(f"\n[Info] Avaliando modelo com dados de treino...")
y_train_prob = model.predict_proba(X_train)[:, 1]
y_pred = (y_train_prob >= 0.5).astype(int)

print(classification_report(y_train, y_pred))
print(f"Acurácia final do modelo: {accuracy_score(y_train, y_pred):.4f}")

print("\nMatriz de Confusão:")
print(confusion_matrix(y_train, y_pred))

print(f"\n[Info] Avaliando modelo com dados de teste...")
y_test_prob = model.predict_proba(X_test)[:, 1]
y_pred = (y_test_prob >= 0.5).astype(int)

print(classification_report(y_test, y_pred))
print(f"Acurácia final do modelo: {accuracy_score(y_test, y_pred):.4f}")

print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))


[Info] Centralizando dados dos participantes...
[Info] Dados centralizados com sucesso. Total de registros: 864000!
    dataReceived      timestamp      ppg  status participant_id
0  1711437352342  1711437340286  2097152       0            P11
1  1711437352342  1711437340327  2097152       0            P11
2  1711437352342  1711437340369  2462981       0            P11
3  1711437352342  1711437340408  2462958       0            P11
4  1711437352342  1711437340447  2462983       0            P11

[Info] Aplicando filtro passa-baixa Butterworth para remover ruído de alta frequência dos sinais PPG...

[Info] Extraindo features HRV...

[Info] Definindo rótulo de anormalidade cardíaca...

[Info] Normalizando as features...

[Info] Separando dados em treino (60%) e teste (40%)...

[Info] Aplicando SMOTE apenas no conjunto de treino...
[Info] Após SMOTE - Classe 0: 1487, Classe 1: 1487

[Info] Treinando modelo com 2974 exemplos...

[Nuvem] Salvando modelo final...
[Nuvem] Modelo salvo em 'Mo